In [ ]:
import pImpactR as impact
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy as copy
import pickle
int = np.vectorize(int)

In [ ]:
rectangular_ = 1
elliptic_ = 2
order = 3

nCore_y = 2
nCore_z = 2
nturn = 100

In [ ]:
emitGeomRMS = 3.3e-6
sigma_K = 2.0e-3
pipe = 0.025

In [ ]:
NL_t = 0.4
NL_L = 1.8
NL_c = 0.01
NL_nu = 0.3
NL_beta = 0.5*NL_L/np.tan(np.pi*NL_nu)

# pipe_info start from NL center

In [ ]:
pipe_info = np.loadtxt('../pipeinfo.in')
index_NLcenter = np.argmin(pipe_info[:,1])

L_tot = pipe_info[-1,0]
s_NLmid = pipe_info[index_NLcenter,0]
print(s_NLmid, L_tot)
print(pipe_info[index_NLcenter-2:index_NLcenter+3,:])

In [ ]:
pipe_info[:,0] = pipe_info[:,0] - s_NLmid
pipe_info[:index_NLcenter,0] = pipe_info[:index_NLcenter,0] + L_tot
pipe_info = np.concatenate((pipe_info[index_NLcenter:,:],pipe_info[:index_NLcenter+1,:]),axis=0)

In [ ]:
pipe_info[-1,0] = L_tot

In [ ]:
plt.figure(figsize=(12,3))
plt.plot(pipe_info[:,0],pipe_info[:,1]*100)
plt.plot(pipe_info[:,0],pipe_info[:,2]*100)
plt.plot(pipe_info[:,0],(pipe_info[:,3]-1)*10,alpha=0.4)
plt.grid()
plt.ylim((0.2,3.5))
plt.xlim((0.0, 40))
plt.minorticks_on()
plt.legend(('x','y'))
plt.xlabel('s (m)')
plt.ylabel('x or y aperture (cm)');

In [ ]:
np.savetxt('pipeinfo.in',pipe_info)

In [ ]:
pipe_info[-4:,:]

# read Lattice

In [ ]:
# beamtmp,lattice1 = impact.readInputFile('test_iota_v8_4_SextOn_NLon.in');
# beamtmp,lattice2 = impact.readInputFile('test_iota_v8_4_SextGoodDA_NLon.in');

# maxS = 0
# for item in lattice1:
#     if item.type == 'multipole_thin':
#         if np.abs(item.KL_sext) > maxS:
#             maxS = np.abs(item.KL_sext)
# for item in lattice2:
#     if item.type == 'multipole_thin':
#         if np.abs(item.KL_sext) > maxS:
#             maxS = np.abs(item.KL_sext)

# print('')
# print('===============================')
# print(' maxS = ', maxS)
# print('===============================')

In [ ]:
beam,lattice = impact.readInputFile('test_iota_v8_4_SextOff_NLon.in')
beam.nCore_y = nCore_y
beam.nCore_z = nCore_z
ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency
g = ke/mass+1.0
bg = np.sqrt(g**2-1.0)
emitN = emitGeomRMS*bg

In [ ]:
ap1_x,ap1_y = 4e-3, 5.5e-3
ang = np.arange(0.0,2*np.pi,2*np.pi/1024)
c = np.cos(ang)
s = np.sin(ang)

# prepare test particles

In [ ]:
nx = 50
testP = 0.0
sig = 0.015
k=0
q_m = beam.multi_charge.q_m[0]
pTest = []
for x in np.linspace(-ap1_x*0.95,ap1_x*0.95,nx):
    for y in np.linspace(-ap1_y*0.95,ap1_y*0.95,nx):
        circ = np.sqrt((x/ap1_x)**2 + (y/ap1_y)**2)
        arcR = np.sqrt(((x-1.3*ap1_x)/ap1_x)**2 + (y/(1.1*ap1_y))**2)
        arcL = np.sqrt(((x+1.3*ap1_x)/ap1_x)**2 + (y/(1.1*ap1_y))**2)
        if circ < 0.95 and y>0:
            if x> 0 and 0.6 < arcR and (circ > 0.75 or  arcR < 0.8):
                pTest.append([x,0.0,y,0.0,0.0,0.0,q_m,0.0,k])
                k=k+1
            elif x<0 and 0.6 < arcL and (circ > 0.75 or  arcL < 0.8):
                pTest.append([x,0.0,y,0.0,0.0,0.0,q_m,0.0,k])
                k=k+1
pTest0 = np.array(pTest)
pTest1 = copy(pTest0)
pTest2 = copy(pTest0)
npt = len(pTest0)


pTest0[:, 5] = -2*sigma_K*ke

pTest1[:,-1] = pTest1[:,-1] + npt

pTest2[:,5] =  2*sigma_K*ke
pTest2[:,-1] = pTest2[:,-1] + 2*npt

pTest = np.concatenate((pTest0,pTest1,pTest2))

impact.writeParticleData(pTest,ke,mass,freq)
beam.n_particles = 3*npt
npt,3*npt

In [ ]:
fig = plt.figure(figsize=[3.3*2,3.3*2])
plt.scatter(pTest[:,0]*1e3, pTest[:,2]*1e3,s=1)
plt.scatter(pTest[:,0]*1e3,-pTest[:,2]*1e3,s=1)
plt.plot(ap1_x*c*1e3,ap1_y*s*1e3,alpha=0.7,c='k',lw=2)
plt.xticks([-5,-2.5,0,2.5,5])
plt.xlim(-ap1_x*1.3*1e3,ap1_x*1.3*1e3)
plt.ylim(-ap1_y*1.3*1e3,ap1_y*1.3*1e3)
plt.grid()

# Prepare lattice

In [ ]:
cleanLat = impact.clearLattice(lattice)
for item in cleanLat:
    if item.type == 'RFkick':
        item.vmax = 0.0
    if 'length' in item:
        item.n_sckick = int(np.ceil(item.length*50))
        item.n_map = 1
    if 'pipe_radius' in item and item.type is not 'dipole':
        item.pipe_radius = pipe

for iNL,elem in enumerate(cleanLat):
    if 'nonlinear' in elem.type:
        break

NL0 = impact.getElem('nonlinear_insert_sliced')

NL0.length = 0.9
NL0.start_position = 0.0
NL0.total_length = 1.8

NL0.tune_advance = 0.3
NL0.strength_t = 0.4
NL0.transverse_scale_c = 0.01

NL0.n_map = 45
NL0.n_sckick = 1

NL1 = copy(NL0)
NL1.start_position = 0.9

cleanLat =  [NL1] + cleanLat[iNL+1:] + cleanLat[:iNL] + [NL0]

# add QFF
cleanLat = impact.addHardEdgeQuad(cleanLat)

In [ ]:
Sext=[]
flagNewSext = False
j=-1
for i,elem in enumerate(cleanLat):
#     print(i,elem.type,flagNewSext)
    if elem.type in ['quad','dipole']:
        flagNewSext = False
    if elem.type == 'multipole_thin':
        if not flagNewSext:
            flagNewSext = True
            j = j+1
            Sext.append(elem)
            elem.sext_family = j
        else:
            cleanLat[i] = Sext[j]

In [ ]:
# test Sext in cleanLat
# for i,item in enumerate(Sext):
#     item.KL_sext = i
    
# for i,elem in enumerate(cleanLat):
#     if elem.type == 'multipole_thin':
#         print(i,elem.KL_sext)

In [ ]:
loop = impact.getElem('loop')

writeIn = impact.getElem('write_raw_ptcl')
writeIn.turn = 1
writeIn.file_id = 100000
writeIn.format_id = 2

writetmp = impact.getElem('write_raw_ptcl')
writetmp.format_id = 2
writetmp.file_id = 100000 + nturn

writeOut = impact.getElem('write_raw_ptcl')
writeOut.format_id = 2
writeOut.turn = nturn
writeOut.file_id = 100000 - nturn


In [ ]:
latticeF = [loop, impact.getElem('pipeinfo'), writeIn] + cleanLat + [writetmp]

In [ ]:
from time import sleep

def getDA_data(SextStr,sec=1.0):
    beam.distribution.distribution_type = 'ReadFile'
    
    # ==== run foward (from NL mid point) ====   
    for i,item in enumerate(Sext):
        item.KL_sext = SextStr[i]
        
    loop.turns = nturn
    writetmp.turn = nturn
    impact.writeInputFile(beam, latticeF)
    impact.run(beam,order=order)
    sleep(sec)
    nptSurvived = impact.readLostAt(-1)
  

    # ==== run backward  ====
    loop.turns = nturn  
    beam.distribution.distribution_type = 'ReadFile_binary'
    beam.distribution.file_id = writetmp.file_id
    
    cleanLat_backward = impact.getInverseLattice(cleanLat)
    latticeB = [loop] + cleanLat_backward + [writeOut]    
      
    impact.writeInputFile(beam, latticeB)
    sleep(sec)
    impact.run(beam,order=order)
    sleep(sec)
    
    
    # collect data
#     pDataIn  = impact.readParticleData(writeIn.file_id,  ke,mass,freq, writeIn.format_id)
    pDataOut = impact.readParticleData(writeOut.file_id, ke,mass,freq, writeOut.format_id)
    
    iSurvived = np.in1d(pTest[:,-1],pDataOut[:,-1])
    pDataIn = pTest[iSurvived,:]
    
    diff = np.sqrt(    (pDataIn[:,0] - pDataOut[:,0])**2/(NL_beta*NL_c*NL_c) + 
                       (pDataIn[:,2] - pDataOut[:,2])**2/(NL_beta*NL_c*NL_c) +
                       (pDataIn[:,1] - pDataOut[:,1])**2*NL_beta/(NL_c*NL_c) + 
                       (pDataIn[:,3] - pDataOut[:,3])**2*NL_beta/(NL_c*NL_c) 
                     ) 
    ind = int(pDataIn[:,-1])
    ind0 = ind<  npt
    ind1 = (  npt<=ind) * (ind<2*npt)
    ind2 = (2*npt<=ind) * (ind<3*npt)
    return ind[ind0],ind[ind1],ind[ind2], diff[ind0], diff[ind1], diff[ind2]

In [ ]:
data = {'x':[],'y':[],'raw':[]}

def cost_func(x):
    sample = getDA_data(x)
    indx = sample[:3]
    diff = sample[3:]
    diffSum0 = np.array([np.sum(diff[i]) for i in range(3)]) 
    diffSum1 = diffSum0+ np.array([np.max(diff[i])*(npt-len(indx[i])) for i in range(3)])
    obj = np.log10(0.15*diffSum1[0] +0.7*diffSum1[1] +0.15*diffSum1[2])
    data['x'].append(x)
    data['y'].append(obj)
    data['raw'].append(sample)
    pickle.dump(data,open('data_local_min.pickle','wb'))
    return obj

# perform local minimization

In [ ]:
from scipy import optimize
result = optimize.minimize(cost_func,np.zeros(12),options = {'disp':True})